In [1]:
import pandas as pd
import re
import numpy as np
import urllib.request
import nltk

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import load_boston
from sklearn import preprocessing
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
df = pd.read_json('News_Dataset.json', lines = True)

In [3]:
#df = df[:50000]

In [4]:
df.tail(5)

,link,headline,category,short_description,authors,date
209522,https://www.huffingtonpost.com/entry/rim-ceo-t...,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,TECH,Verizon Wireless and AT&T are already promotin...,"Reuters, Reuters",2012-01-28
209523,https://www.huffingtonpost.com/entry/maria-sha...,Maria Sharapova Stunned By Victoria Azarenka I...,SPORTS,"Afterward, Azarenka, more effusive with the pr...",,2012-01-28
209524,https://www.huffingtonpost.com/entry/super-bow...,"Giants Over Patriots, Jets Over Colts Among M...",SPORTS,"Leading up to Super Bowl XLVI, the most talked...",,2012-01-28
209525,https://www.huffingtonpost.com/entry/aldon-smi...,Aldon Smith Arrested: 49ers Linebacker Busted ...,SPORTS,CORRECTION: An earlier version of this story i...,,2012-01-28
209526,https://www.huffingtonpost.com/entry/dwight-ho...,Dwight Howard Rips Teammates After Magic Loss ...,SPORTS,The five-time all-star center tore into his te...,,2012-01-28


In [5]:
#df.isna().sum() 

In [6]:
df = df.drop_duplicates() 

In [7]:
#df.nunique()

In [8]:
#df.info()

In [9]:
cols = ['headline', 'short_description']
df['combined'] = df[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
df.combined[:2]

0    Over 4 Million Americans Roll Up Sleeves For O...
1    American Airlines Flyer Charged, Banned For Li...
Name: combined, dtype: object

In [10]:
words = set(nltk.corpus.words.words())
def preprocess_text(text):
    """ Apply any preprocessing methods"""
    str1 = " "
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = " ".join(w for w in nltk.wordpunct_tokenize(text) \
         if w.lower() in words or not w.isalpha())
#     stop_words = set(stopwords.words('english'))
#     word_tokens = word_tokenize(text)
#     filtered = [w for w in word_tokens if not w.lower() in stop_words]
    
#     com = str1.join(filtered)
    return text

df["combined"] = df.combined.apply(preprocess_text)
df.combined[2284]

'deadly coral disease linked to ship wastewater_the in coral in the are linked by to the stony coral tissue loss disease which may be triggered by ship traffic'

In [11]:
vectorizer = CountVectorizer(stop_words='english')
documents_vectorized = vectorizer.fit_transform(df['combined'])
vocabulary = vectorizer.get_feature_names_out()
vocabulary

array(['00', '000', '0000', ..., 'čechomor_čechomor', 'κάιρο_',
       'ᵒᴥᵒᶅ_the'], dtype=object)

In [12]:
print ('We have a {} document corpus with a {} term vocabulary'.format(*documents_vectorized.shape))

We have a 209514 document corpus with a 172143 term vocabulary


In [13]:
dataframe = pd.DataFrame(documents_vectorized.toarray(), columns=vocabulary)
doc_ids = dataframe.index.values


In [14]:
dataframe[:5]

,00,000,0000,000064,0000_remember,0001_were,0002,000foot,001,0010,...,zzzs_and,zzzs_rather,zzzs_snooze,½year,émigrés_the,étienne_william,ñ_director,čechomor_čechomor,κάιρο_,ᵒᴥᵒᶅ_the
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
def BM25_IDF_df(df):
  """
  This definition calculates BM25-IDF weights before hand as done last week
  """

  dfs = (df > 0).sum(axis=0)
  N = df.shape[0]
  idfs = -np.log(dfs / N)
  
  k_1 = 1.2
  b = 0.8
  dls = df.sum(axis=1) 
  avgdl = np.mean(dls)

  numerator = np.array((k_1 + 1) * df)
  denominator = np.array(k_1 *((1 - b) + b * (dls / avgdl))).reshape(N,1) \
                         + np.array(df)

  BM25_tf = numerator / denominator

  idfs = np.array(idfs)

  BM25_score = BM25_tf * idfs
  return pd.DataFrame(BM25_score, columns=vocabulary)


In [16]:
bm25_df = BM25_IDF_df(dataframe[:1000])

# for i in range(1000, 209514, 1000):
#     bm25_df1 = BM25_IDF_df(dataframe[:i])  # a dataframe with BM25-idf weights
#     bm25_df.append(bm25_df1, ignore_index = True)
    
for i in range(1000, 4000, 1000):
    bm25_df1 = BM25_IDF_df(dataframe[:i])  # a dataframe with BM25-idf weights
    bm25_df.append(bm25_df1, ignore_index = True)

bm25_df[:5]

/Users/usamaal-attia/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/var/folders/gf/9676_22j2d94hj9mxjytm1qw0000gn/T/ipykernel_6508/309118988.py:23: RuntimeWarning: invalid value encountered in multiply
  BM25_score = BM25_tf * idfs
/Users/usamaal-attia/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/var/folders/gf/9676_22j2d94hj9mxjytm1qw0000gn/T/ipykernel_6508/309118988.py:23: RuntimeWarning: invalid value encountered in multiply
  BM25_score = BM25_tf * idfs
/var/folders/gf/9676_22j2d94hj9mxjytm1qw0000gn/T/ipykernel_6508/1891289038.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bm25_df.append(bm25_df1, ignore_index = True)
/Users/usamaal-a

,00,000,0000,000064,0000_remember,0001_were,0002,000foot,001,0010,...,zzzs_and,zzzs_rather,zzzs_snooze,½year,émigrés_the,étienne_william,ñ_director,čechomor_čechomor,κάιρο_,ᵒᴥᵒᶅ_the
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
queries = dict(enumerate(['ship wreck',
                          'little boat']))

def retrieve_ranking(query, bm25_df):
  q_terms = query.split(' ')
  q_terms_only = bm25_df[q_terms]
  score_q_d = q_terms_only.sum(axis=1)
  return sorted(zip(bm25_df.index.values, score_q_d.values),
                key = lambda tup:tup[1],
                reverse=True)


# Let's look at the first few scores for our query and document combinations:
for count, query in enumerate(queries.values()):
  print(f'Query {count}: {query}')
  print('')
  results = retrieve_ranking(query, bm25_df)
  print('')

Query 0: ship wreck


Query 1: little boat




In [18]:
final = []
for i in results[:10]:
    final.append(i[0])
print(final)

[797, 237, 581, 832, 439, 766, 375, 671, 0, 1]


In [37]:
links = df.loc[final, 'link']
links

797    https://www.huffpost.com/entry/japan-shiretoko...
237    https://www.huffpost.com/entry/inflation-reduc...
581    https://www.huffpost.com/entry/kim-kardashian-...
832    https://www.huffpost.com/entry/lizzo-youre-bea...
439    https://www.huffpost.com/entry/funniest-tweets...
766    https://www.huffpost.com/entry/samantha-bee-ma...
375    https://www.huffpost.com/entry/dave-coulier-yo...
671    https://www.huffpost.com/entry/health-insuranc...
0      https://www.huffpost.com/entry/covid-boosters-...
1      https://www.huffpost.com/entry/american-airlin...
Name: link, dtype: object

In [20]:

def precision_at_k(query_id, k=5):
  """This function considers the k top ranking documents."""
  doc_ranking = retrieve_ranking(queries[query_id], bm25_df)

  # take only the document id, rather than score
  retrieved = [doc[0] for doc in doc_ranking[:k]]

  TP = ...  # number of true positives
  FP = ...  # number of false positives 

  precision = ...

  return TP, FP, precision


# Let's see what we get when we consider the top 5 ranking documents:
def print_precision_for_all_queries(k=5):
  for query_id, query in queries.items():
    TP, FP, precision = precision_at_k(query_id, k=k) 
    print(f'retrieved query "{query}" with precision @ {k}: {precision} (TP: {TP}, FP: {FP})')

In [35]:
headline = df.loc[final, 'headline']
headline

797    10 Found Dead After Japan Tour Boat Carrying 2...
237    Democrats’ Inflation Reduction Act Expected To...
581    Kim Kardashian Discusses Pete Davidson's 'BDE'...
832    Determined Lizzo Tries To Break The 'Bitch' Re...
439    The Funniest Tweets From Women This Week (June...
766    Samantha Bee Stunned By 'Gossipy Little Bitch'...
375    'Oh No': Dave Coulier Recalls Hearing Alanis M...
671    Democrats Have Little Time To Avert An Electio...
0      Over 4 Million Americans Roll Up Sleeves For O...
1      American Airlines Flyer Charged, Banned For Li...
Name: headline, dtype: object

In [39]:
disc = df.loc[final, 'short_description']
disc

797    The boat was sailing in an area known as a dif...
237    The bill was also touted as lowering the defic...
581    The reality star finally goes there on Hulu's ...
832    She also addressed rumors that she's "dating e...
439    "Sending friends tweets I think they might lik...
766    "Damn! Mark Meadows, you are messy,” she said....
375    The "Full House" star revealed one big clue th...
671    A subsidy boost for the Affordable Care Act th...
0      Health experts said it is too early to predict...
1      He was subdued by passengers and crew when he ...
Name: short_description, dtype: object